In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
target = fep_control.MountainCarStep(state_dim=2, batch_shape=(1,), trainable=False)
proposal = fep_control.RecognitionStep(state_dim=2, observation_dim=2, batch_shape=(1,), discrete_actions=False, name='MountainCarStep', trainable=True)
agent = importance.propose(target, proposal)

In [5]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [6]:
theta, graph, log_weight = active.active_inference(agent, 'MountainCarContinuous-v0', lr=1e-4, episodes=10, episode_length=1000, dream=False)

05/29/2019 11:36:44 ELBO=-5.58942676e+03 at episode 0 of length 922
05/29/2019 11:37:00 ELBO=-5.46105566e+03 at episode 1 of length 713
05/29/2019 11:37:12 ELBO=-5.63968164e+03 at episode 2 of length 717
05/29/2019 11:37:23 ELBO=-4.90460156e+03 at episode 3 of length 1000
05/29/2019 11:37:34 ELBO=-6.31630078e+03 at episode 4 of length 71
05/29/2019 11:37:46 ELBO=-6.07073291e+03 at episode 5 of length 405
05/29/2019 11:37:57 ELBO=-8.51472852e+03 at episode 6 of length 76
05/29/2019 11:38:14 ELBO=-6.92935693e+03 at episode 7 of length 540
05/29/2019 11:38:29 ELBO=-5.79679980e+03 at episode 8 of length 474
05/29/2019 11:38:40 ELBO=-9.33758008e+03 at episode 9 of length 613


In [7]:
log_weight

tensor([-9337.5801], grad_fn=<AddBackward0>)

In [8]:
theta, graph, log_weight = active.active_inference_test(agent, 'MountainCarContinuous-v0', online_inference=False, iterations=1000)

In [9]:
theta

(tensor([[ 0.5688, -0.1106]], grad_fn=<AddBackward0>),
 tensor([[6.8694]], grad_fn=<ExpandBackward>))

In [10]:
log_weight

tensor([-631.0915], grad_fn=<AddBackward0>)

In [11]:
torch.save(agent, 'examples/fep_control/fep_mountain_car_agent.dat')